In [1]:
import pandas as pd
import pingouin as pg

In [2]:
df = pd.read_csv('/Users/ira/Downloads/REALEC_Inspector/data/feature_result.csv')
df.head()

,av_depth,max_depth,min_depth,num_acl,num_rel_cl,num_advcl,num_sent,num_tok,av_tok_before_root,av_len_sent,...,punct_mistakes_pp,punct_mistakes_because,punct_mistakes_but,punct_mistakes_compare,million_mistake,side_mistake,name,text,class,type
0,3.555556,6,2,2,0,1,18,244,4.388889,13.555556,...,0,0,0,0,0,0,./data/exam/exam2017/OBy_146_1,This two line graphs illustrates monthly avera...,60,1
1,5.428571,10,4,1,3,1,7,175,6.285714,25.000000,...,0,0,0,0,0,0,./data/exam/exam2017/EGe_15_1,"﻿We have two graphics, which show us the popul...",65,1
2,3.812500,6,0,5,0,13,16,292,4.562500,18.250000,...,0,0,1,0,0,0,./data/exam/exam2017/DOv_2_2,More and more young people are stunding on the...,60,2
3,4.333333,7,2,3,3,6,12,221,6.666667,18.416667,...,0,2,0,0,0,0,./data/exam/exam2017/EGe_61_2,﻿Nowdays we can notice the tendency that large...,65,2
4,4.250000,5,3,2,3,0,8,160,7.125000,20.000000,...,0,0,1,0,0,0,./data/exam/exam2017/NMya_48_1,The graph gives information about level of une...,55,1


In [3]:
targets = []
for t in df['class']:
    if int(t) >= 70:
        targets.append('best')
    else:
        targets.append('not_best')
df['class'] = targets

# TTR

The general rules of thumb given by Cohen and Miles & Shevlin (2001) for analyzing eta-squared:

* Small effect: 0.01
* Medium effect: 0.06
* Large effect: 0.14

- ttr

In [33]:
ttr = pg.anova(data=df, dv='ttr', between='class', detailed=True)
p_ttr = ttr['p-unc'][0]
np2_ttr = ttr['np2'][0]
ttr

,Source,SS,DF,MS,F,p-unc,np2
0,class,0.002,1,0.002,0.602,0.437722,0
1,Within,12.841,3440,0.004,-,-,-


- cttr

In [34]:
cttr = pg.anova(data=df, dv='corrected_ttr', between='class', detailed=True)
p_cttr = cttr['p-unc'][0]
np2_cttr = cttr['np2'][0]
cttr

,Source,SS,DF,MS,F,p-unc,np2
0,class,61.344,1,61.344,97.415,1.12176e-22,0.028
1,Within,2166.246,3440,0.630,-,-,-


- rttr

In [35]:
rttr = pg.anova(data=df, dv='root_ttr', between='class', detailed=True)
p_rttr = rttr['p-unc'][0]
np2_rttr = rttr['np2'][0]
rttr

,Source,SS,DF,MS,F,p-unc,np2
0,class,122.689,1,122.689,97.415,1.12176e-22,0.028
1,Within,4332.493,3440,1.259,-,-,-


- logttr

In [36]:
log_ttr = pg.anova(data=df, dv='log_ttr', between='class', detailed=True)
p_log_ttr = log_ttr['p-unc'][0]
np2_log_ttr = log_ttr['np2'][0]
log_ttr

,Source,SS,DF,MS,F,p-unc,np2
0,class,0.005,1,0.005,9.39,0.00219916,0.003
1,Within,1.750,3440,0.001,-,-,-


- uber

In [39]:
uber = pg.anova(data=df, dv='uber_ttr', between='class', detailed=True)
p_uber_ttr = uber['p-unc'][0]
np2_uber_ttr = uber['np2'][0]
uber

,Source,SS,DF,MS,F,p-unc,np2
0,class,7.092278e+03,1,7092.278,0.263,0.608303,0
1,Within,9.287109e+07,3440,26997.411,-,-,-


- d

In [40]:
d = pg.anova(data=df, dv='d', between='class', detailed=True)
p_d = d['p-unc'][0]
np2_d = d['np2'][0]
d

,Source,SS,DF,MS,F,p-unc,np2
0,class,0.079,1,0.079,29.181,7.0394e-08,0.008
1,Within,9.359,3440,0.003,-,-,-


In [42]:
result_ttr = pd.DataFrame(data = {'feature': ['ttr',
                                              'corrected_ttr',
                                              'root_ttr',
                                              'log_ttr',
                                              'uber_ttr',
                                              'd'],
                                  'p-value': [round(p_ttr, 4), round(p_cttr, 4), round(p_rttr, 4),
                                              round(p_log_ttr, 4), round(p_uber_ttr, 4), round(p_d, 4)],
                                  'eta-squared': [round(np2_ttr, 4), round(np2_cttr, 4), round(np2_rttr, 4),
                                                  round(np2_log_ttr, 4), round(np2_uber_ttr, 4), round(np2_d, 4)]})
result_ttr

,feature,p-value,eta-squared
0,ttr,0.4377,0.000
1,corrected_ttr,0.0000,0.028
2,root_ttr,0.0000,0.028
3,log_ttr,0.0022,0.003
4,uber_ttr,0.6083,0.000
5,d,0.0000,0.008


# VS

In [46]:
def get_info(feature):
    f = pg.anova(data=df, dv=feature, between='class', detailed=True)
    p = f['p-unc'][0]
    np2 = f['np2'][0]
    return p, np2

In [47]:
p_vs, np2_vs = get_info('vs')
p_corrected_vs, np2_corrected_vs = get_info('corrected_vs')
p_squared_vs, np2_squared_vs = get_info('squared_vs')

In [48]:
result_vs = pd.DataFrame(data = {'feature': ['vs',
                                              'corrected_vs',
                                              'squared_vs'],
                                  'p-value': [round(p_vs, 4), round(p_corrected_vs, 4), round(p_squared_vs, 4)],
                                  'eta-squared': [round(np2_vs, 4), round(np2_corrected_vs, 4), round(np2_squared_vs, 4)]})
result_vs

,feature,p-value,eta-squared
0,vs,0.7855,0.000
1,corrected_vs,0.0059,0.002
2,squared_vs,0.0039,0.002


# VV

In [62]:
p_vvi, np2_vvi = get_info('vvi')
p_vvii, np2_vvii = get_info('vvii')
p_squared_vv, np2_squared_vv = get_info('squared_vv')
p_corrected_vv, np2_corrected_vv = get_info('corrected_vv')

In [63]:
result_vs = pd.DataFrame(data = {'feature': ['vvi',
                                             'vvii',
                                             'squared_vv',
                                             'corrected_vv'],
                                  'p-value': [round(p_vvi, 4), round(p_vvii, 4),
                                              round(p_squared_vv, 4), round(p_corrected_vv, 4)],
                                  'eta-squared': [round(np2_vvi, 4), round(np2_vvii, 4),
                                                  round(np2_squared_vv, 4), round(np2_corrected_vv, 4)]})
result_vs

,feature,p-value,eta-squared
0,vvi,0.1823,0.001
1,vvii,0.0325,0.001
2,squared_vv,0.0000,0.027
3,corrected_vv,0.0000,0.022


# All

In [72]:
features = df.columns.to_list()
features.remove('class')
features.remove('text')
features.remove('name')
features.remove('type')
features.remove('side_mistake')
d = {'feature': [], 'p-value': [], 'eta-squared': []}
for f in features:
    p, np2 = get_info(f)
    d['feature'].append(f)
    d['p-value'].append(round(p, 4))
    d['eta-squared'].append(round(np2, 4))
result = pd.DataFrame(data=d)

In [73]:
result

,feature,p-value,eta-squared
0,av_depth,0.0000,0.026
1,max_depth,0.0000,0.016
2,min_depth,0.0004,0.004
3,num_acl,0.0000,0.031
4,num_rel_cl,0.0000,0.013
5,num_advcl,0.0000,0.026
6,num_sent,0.0207,0.002
7,num_tok,0.0000,0.032
8,av_tok_before_root,0.0000,0.005
9,av_len_sent,0.0000,0.024


In [74]:
result.to_csv('ANOVA.csv', index=False)